<a href="https://colab.research.google.com/github/Luccascarpa/PO-II/blob/main/Notebook_P1_Programa%C3%A7%C3%A3o_Din%C3%A2mica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import plotly.express as px
import plotly.graph_objects as go
import time
import functools

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
class interactive_board():
  def __init__(self, board, fase, posicao, desafio=1):
    self.board_original = board
    self.desafio = desafio
    self.fase = fase
    self.posicao = posicao
    
    self.f = None#go.FigureWidget(px.imshow(self.board_original.copy()>0.5))
    self.new_premio()

  def new_premio(self):
    if self.desafio in [1,2]:
      self.premio_posicao = np.random.choice(np.arange(posicoes))
      self.premio_fase = fases-1
    else:
      self.premio_posicao = np.random.choice(np.arange(posicoes))
      self.premio_fase = np.random.choice(np.arange(fases),
                                          p=[1/(fases-1) if i!=self.fase else 0 for i in range(fases)])
    
    self.update_plot()

  def update_plot(self):
    if self.desafio in [2, 4]:
      new_board = 1*(np.random.rand(posicoes, fases)<self.board_original.copy())
    else:
      new_board = 1*(0.5<self.board_original.copy())
    
    new_board[self.premio_posicao, self.premio_fase] = 2
    if new_board[self.posicao, self.fase]==1:
      new_board[self.posicao, self.fase] = -2
    else:
      new_board[self.posicao, self.fase] = -1
    if self.f is None:
      self.f = go.FigureWidget(px.imshow(new_board, range_color=[-2, 2]))
    else:
      self.f.data[0]['z'] = new_board
    return True

  def update(self, fase, posicao):
    self.fase = fase
    self.posicao = posicao
    if fase==self.premio_fase and posicao==self.premio_posicao:
      self.new_premio()

    self.update_plot()

def criar_tabuleiro(fases, posicoes):
  board = np.maximum(np.random.rand(posicoes, fases)-0.25, 0)
  return board

In [ ]:
fases = 20
posicoes = 5
board = criar_tabuleiro(fases, posicoes)

In [ ]:
board.T

array([[7.10322354e-01, 4.81872818e-01, 3.14103899e-01, 2.98536465e-01,
        3.85352774e-01],
       [3.84639994e-01, 0.00000000e+00, 4.44480680e-01, 3.54173941e-01,
        1.82810978e-01],
       [0.00000000e+00, 6.03389127e-01, 0.00000000e+00, 0.00000000e+00,
        4.60393081e-01],
       [6.56140390e-02, 6.49905230e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [2.41070677e-01, 0.00000000e+00, 1.95731880e-01, 4.44453462e-02,
        0.00000000e+00],
       [3.88238952e-02, 2.05331378e-02, 5.77312947e-01, 0.00000000e+00,
        0.00000000e+00],
       [1.82703879e-02, 2.05230388e-01, 5.88468597e-01, 1.86280196e-01,
        1.92317307e-01],
       [1.42957189e-01, 7.72032158e-03, 4.28759562e-01, 2.83116527e-01,
        1.14624399e-01],
       [5.74868577e-01, 3.34355276e-01, 3.48183882e-01, 0.00000000e+00,
        6.29202214e-01],
       [8.54459311e-02, 6.96366053e-01, 3.52028687e-01, 3.94289590e-01,
        2.04162516e-02],
       [6.22939439e-01, 6.3416

In [ ]:
posicao = 3
fase = 0
iter_board_1 = interactive_board(board, fase, posicao, desafio=1)
#iter_board.f
board_positions = iter_board_1.f['data'][0]['z']
board_positions = pd.DataFrame(board).replace(2,1)

In [ ]:
def custo_a_b(a, b, pos_A, pos_B, x_destino_final=iter_board_1.premio_posicao, y_destino_final=iter_board_1.premio_fase, backwards=False):
    """Dadas duas casas do tabuleiro, A e B, e sendo elas casas de estágios
    consecutivos, isto é, coluna de A é igual a coluna de B menos um, a fun-
    ção retorna o custo de deslocamento entre as casas."""
    
    # verifica se as posições passadas estão no tabuleiro

    if pos_A[0] >=5:
        print("Position A out of bounds")
        return None
    if pos_B[0] >=5:
        print("Position B out of bounds")
        return None
    if pos_A[1] >=20:
        print("Position A out of bounds")
        return None
    if pos_B[1] >=20:
        print("Position B out of bounds")

    # verifica se os valores são consecutivos
    if backwards:
        nao_consecutivos = pos_B[1] - pos_A[1] != -1
    else:
      nao_consecutivos = pos_B[1] - pos_A[1] != 1
    if nao_consecutivos:
        print("Favor, entre valores consecutivos.")
        return None

    # define os valores de cada uma das cores

    laranja = 1
    rosa = 0

    # calcula, baseado nos parâmetros

    if a == laranja and b == rosa:
        custo = 30
    else:
        deslocamento_vertical = abs(pos_A[0] - pos_B[0])
        custo = 1 + deslocamento_vertical**2

    # casos do último estágio, onde o valor final TEM QUE SER o x_destino_final e o y_destino_final
    
    if pos_B[1] == y_destino_final and pos_B[0] != x_destino_final:
        custo = np.inf
    
    return custo

In [ ]:
# define a matriz de entrada do caminho
caminho_default = pd.Series([1e10 for i in range(20)],index = range(20), dtype=float)
caminhos=[]
tabuleiros ={}

def gera_boards_positions(tab=1):
  if tab==1: 
    return pd.DataFrame(iter_board_1.f['data'][0]['z'])
  elif tab==2:
    return pd.DataFrame(iter_board_2.f['data'][0]['z'])
  elif tab==3:
    return pd.DataFrame(iter_board_3.f['data'][0]['z'])

def get_x_y_finais(tab=1):
  if tab==1: 
    return iter_board_1.premio_posicao, iter_board_1.premio_fase
  elif tab==2:
    return iter_board_2.premio_posicao, iter_board_2.premio_fase
  elif tab==3:
    return iter_board_3.premio_posicao, iter_board_3.premio_fase


@functools.lru_cache(maxsize=None)
def cam_min(inicial, final, estagio=0, tab=1, registra_tabuleiros=False):
    """
    inicial (tuple): posição inicial no tabuleiro
    final (tuple): posição final no tabuleiro
    estagio (int): estagio, ou seja, coluna, da posição inicial no tabuleiro
    caminho (pd.Series): vetor com os caminhos registrados até o momento
    """

    x_final, y_final = get_x_y_finais(tab=tab)

    # pega as posições do tabuleiro do momento
    board_positions = gera_boards_positions(tab=tab)
    board_positions = board_positions.replace(2,0)

    # preenche a primeira posição no estado inicial
    if estagio == 0:
        caminho_default[0] = inicial[0]
        tabuleiros[0] = board_positions

    # se o estagio inicial for igual ao final

    if inicial[1] == final[1]:
        return 0, caminhos
    
    custo_min = np.inf
    
    prox_estagio = estagio + 1
    
    possiveis_casas = board_positions[prox_estagio]

    for row, casa in enumerate(possiveis_casas):

            position_a = inicial
            position_b = (row, prox_estagio)

            cor_a = board_positions.loc[inicial[0], inicial[1]]
            cor_b = casa
            
            custo_jogada = custo_a_b(cor_a, cor_b, position_a, position_b, x_destino_final=x_final, y_destino_final=y_final)

            custo_prox_final, *_ = cam_min(position_b, final, estagio = prox_estagio)

            custo = custo_prox_final + custo_jogada

            if custo < custo_min:
                custo_min = custo
                caminho_default[estagio+1] = row
                caminhos.append(caminho_default)
                if registra_tabuleiros == True:
                    tabuleiros[estagio+1] = board_positions
    
    caminho = caminhos[-1]
    caminho = caminho.head(y_final+1)
    return custo_min, caminho

In [ ]:
### DESAFIO 1

"""

Para a resolução do Desafio 1, basta aplicar a função cam_min, para obter o custo mínimo. Além
do custo, é interessante saber o caminho. Para isso, é necessário definir um objeto Series com suas
chaves iguais às fases, que será alimentado com a melhor posição para aquela fase. 
    Obs: o caminho foi definido previamente à função.

"""

inicio = (3,0)
final = (iter_board_1.premio_posicao,iter_board_1.premio_fase)
custo_minimo, caminho = cam_min(inicio, final)

print(f"""\n{10*'='} DESAFIO 1 - Tabuleiro Determinístico e Objetivo Único {10*'='}\n\nCaminho mínimo no Desafio 1, saindo do ponto {inicio} para o ponto {final}: {custo_minimo}.\n\nCaminho percorrido: \n{caminho}""")


========== DESAFIO 1 - Tabuleiro Determinístico e Objetivo Único ==========

Caminho mínimo no Desafio 1, saindo do ponto (3, 0) para o ponto (1, 19): 21.

Caminho percorrido: 
0     3.0
1     3.0
2     3.0
3     3.0
4     3.0
5     3.0
6     3.0
7     3.0
8     3.0
9     2.0
10    2.0
11    2.0
12    2.0
13    2.0
14    2.0
15    3.0
16    0.0
17    2.0
18    0.0
19    1.0
dtype: float64


In [ ]:
iter_board_1.f

FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>c…

In [ ]:
## Andar o tabuleiro no desafio 1

def anda_tabuleiro_1(caminho):
  """Essa função recebe de entrada um caminho 
  e roda ele no último tabuleiro plotado"""

  caminho = caminho.astype(int).to_list()

  for fase, posicao in enumerate(caminho):
    iter_board_1.update(fase, posicao)

anda_tabuleiro_1(caminho)

In [ ]:
posicao = 3
fase = 0
iter_board_2 = interactive_board(board, fase, posicao, desafio=2)
iter_board_2.f

FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>c…

In [ ]:
## Andar o tabuleiro no desafio 2

def anda_tabuleiro_2(caminho_inicial):
  """Essa função recebe de entrada um caminho 
  e roda ele no último tabuleiro plotado"""

  caminho_inicial = caminho_inicial.astype(int).to_list()

  inicial = (3,0)
  final = (iter_board_2.premio_posicao, 19)

  registro_caminhos = pd.DataFrame([])

  for fase in range(19):
    # calcula o caminho e o preço
    custo, caminho = cam_min(inicial, final, estagio=fase, tab=2, registra_tabuleiros=True)
    
    # registra o caminho
    registro_caminhos[f'Fase {fase}'] = caminho

    # pega a próxima fase e posição
    fase += 1
    posicao = int(caminho[fase])

    # atualiza no gráfico
    iter_board_2.update(fase, posicao)
    

    # roda o código que encontra o caminho novamente
    inicial = (posicao, fase)
    custo, caminhos = cam_min(inicial, final, estagio=fase, tab=2)

    # coloca como NaN todos os caminhos prévios ao estágio
    for i in range(fase):
      caminho[i] = np.NaN

    if fase == 19:
      registro_caminhos[f'Fase {fase}'] = caminho
  
  # tira a diagonal da matriz para achar o caminho percorrido

  caminho_otimo = pd.Series([], dtype=float)
  for i in range(20):
    casa = registro_caminhos.iloc[i,i]
    caminho_otimo[i] = casa
  return caminho_otimo


caminho = anda_tabuleiro_2(caminho)

In [ ]:
def get_custo_tab2(caminho, tabuleiros):
    """O dicionário "tabuleiros", que foi registrado quando se percorreu a função 
    compila todos os caminhos que foram registrados à medida que a função foi cha-
    mada, e será usado para o cálculo do custo de deslocamento."""

    custo = 0
    for i in range(18):
        curr_tab = tabuleiros[i]
        posicao_a = (caminho[i], i)
        posicao_b = (caminho[i+1],i+1)

        cor_a = curr_tab.loc[caminho[i], i]
        cor_b = curr_tab.loc[caminho[i+1],i+1]

        custo_jogada = custo_a_b(cor_a,cor_b,posicao_a, posicao_b)

        custo += custo_jogada

    return custo

custo_minimo = get_custo_tab2(caminho, tabuleiros)

In [ ]:
### DESAFIO 2

"""

Para a resolução do Desafio 2, é necessário recalcular o caminho ótimo toda vez que
o jogador troca de casa, como se o GPS recebesse umna atualização e precisasse mudar 
sua rota. Para isso, a função calcula o caminho míimo, registra ele, anda uma casa,
reclacula o caminho, registra ele, anda mais uma casa, e assim sucessivamente.
  Obs: para que os códigos do desafio 2 funcionem, é preciso que tudo seja rodado
sequência, ou seja, com o atributo run all.


"""

inicio = (3,0)
final = (iter_board_2.premio_posicao,19)

print(f"""\n{15*'='} DESAFIO 2 - Tabuleiro Estocástico e Objetivo Único {15*'='}\n\nCaminho mínimo no Desafio 2, saindo do ponto {inicio} para o ponto {final}: {custo_minimo}.\n\nCaminho percorrido: \n{caminho}""")


=============== DESAFIO 2 - Tabuleiro Estocástico e Objetivo Único ===============

Caminho mínimo no Desafio 2, saindo do ponto (3, 0) para o ponto (0, 19): 77.0.

Caminho percorrido: 
0     3.0
1     3.0
2     3.0
3     3.0
4     3.0
5     3.0
6     3.0
7     2.0
8     2.0
9     2.0
10    2.0
11    2.0
12    2.0
13    2.0
14    2.0
15    2.0
16    2.0
17    2.0
18    2.0
19    2.0
dtype: float64


In [ ]:
def getIndexes(dfObj, value):
     
    # Empty list
    listOfPos = []
     
    # isin() method will return a dataframe with
    # boolean values, True at the positions   
    # where element exists
    result = dfObj.isin([value])
     
    # any() method will return
    # a boolean series
    seriesObj = result.any()
 
    # Get list of column names where
    # element exists
    columnNames = list(seriesObj[seriesObj == True].index)
    
    # Iterate over the list of columns and
    # extract the row index where element exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
 
        for row in rows:
            listOfPos.append((row, col))
             
    # This list contains a list tuples with
    # the index of element in the dataframe
    return listOfPos

In [ ]:
## DESAFIO 3

posicao = 3
fase = 0
iter_board_3 = interactive_board(board, fase, posicao, desafio=3)
iter_board_3.f

FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>c…

In [ ]:
def segunda_etapa_desafio_3():
    """Essa função passa de um ponto arbitrário a outro do tabuleiro, 
    registrando o custo desse caminho, em três possíveis casos, a di-
    reita, à esquerda ou na mesma coluna"""

    current_position, obj_2, caminho1 = primeira_etapa_desafio_3()

    fase_atual = current_position[1]
    fase_destino = obj_2[1]

    posicao_atual = current_position[0]
    posicao_destino = obj_2[0]

    if fase_atual < fase_destino:
      custo, caminho2 = cam_min(current_position, obj_2, estagio=fase_atual)
      for i in range(fase_atual):
        caminho2[i] = np.NaN
      caminho2 = caminho2.head(fase_destino+1)
    
    if fase_atual > fase_destino:
      custo, caminho2 = cam_min(obj_2, current_position, estagio=fase_destino)
      for i in range(fase_destino):
        caminho2[i] = np.NaN
      caminho2 = caminho2.head(fase_atual+1)
      caminho2 = caminho2.iloc[::-1]

    if fase_atual == fase_destino:
      caminho2 = 0
    
    print("Caminho 1:", caminho1)
    print("Caminho 2:", caminho2)


segunda_etapa_desafio_3()


Caminho 1: 0    3.0
1    3.0
2    4.0
3    4.0
4    4.0
5    4.0
6    4.0
dtype: float64
Caminho 2: 0


In [ ]:
def primeira_etapa_desafio_3(inicial = (3,0)):
    """Essa função retorna a posição final e o novo objetivo e calcula 
    o caminho para chegar ao primeiro objetivo traçado no tabuleiro 3"""

    # calcula o custo e o caminho para andara até o primeiro objetivo 

    obj_2 = getIndexes(board_positions, 2)[0]
    custo1, caminho1 = cam_min(inicial, obj_1, tab=3)
    list_caminho1 = caminho1.astype(int).to_list()

    # anda no tabuleiro

    for fase, posicao in enumerate(list_caminho1):
        iter_board_3.update(fase, int(posicao))
    board_positions = gera_boards_positions(tab=3)
    obj_2 = getIndexes(board_positions, 2)[0]

    current_position = obj_1
    
    return current_position, obj_2, caminho1

((1, 9), (1, 10), 0    3.0
 1    3.0
 2    3.0
 3    3.0
 4    3.0
 5    3.0
 6    3.0
 7    3.0
 8    3.0
 9    1.0
 dtype: float64)

In [ ]:
obj_1 = get_x_y_finais(tab=3)

cam_min(obj_1,(3,0), estagio=obj_1[1], tab=3, backwards=True)

(0, 17) (0, 16)
Favor, entre valores consecutivos.
(0, 16) (0, 17)
Favor, entre valores consecutivos.
(0, 17) (0, 18)
Favor, entre valores consecutivos.
(0, 18) (0, 19)


KeyError: ignored